In [ ]:
#-*- coding:utf-8 -*-
from commonTool import *
from config import *


# Parameter 설정
set_enc_param('Ti9RYkVTZUtteTNHajA1U29qTkd3UT09')
print('enc_param', enc_param_value, flush=True)
print('codeFile', codePathFile, flush=True)



def doFetch(codeFile, outPath, fetchType, funcGet):
    print(fetchType, 'doing...', flush=True)
    deli = '\t'
    # keyNames = ['ACCODE', 'ACC_NM', 'UNT_TYP', 'P_ACCODE', 'DATA1', 'DATA2', 'DATA3', 'DATA4', 'DATA5']
    valueColumns = ['DATA1', 'DATA2', 'DATA3', 'DATA4', 'DATA5']
    
    f = open(codeFile, 'r')
    
    accCheck = {}
    accNames = []
    accItems = []
    
    dataSet = []
    
    noLine = 0
    while True:
        line = f.readline()
        if not line:
            break

        noLine += 1
        code = line.strip()
        if len(code) <= 0:
            continue

        print('fetching', code, flush=True)
        resp = funcGet(code)

        if resp.status_code == 200:
            # print(resp.text, flush=True)
            obj = json.loads(resp.text)
            if obj is None: continue
            
            # DATA1 ~ DATA5의 기준 년월
            yearName = [T[0:T.find('<')] for T in obj['YYMM']][0:len(valueColumns)]

            data = obj['DATA']
            if data is None:
                continue

            items = [extendList([code, title], len(accNames) + 2) for title in yearName] # DATA1, ..., DATA4
            
            for rec in data:
                if not rec['ACCODE'] in accCheck:
                    accCheck[rec['ACCODE']] = len(accNames)
                    accNm = rec['ACC_NM'].lstrip('.').lstrip('*')
                    accNames.append(accNm if rec['P_ACCODE'] is None else accNames[accCheck[rec['P_ACCODE']]] + '/' + accNm)
                    accItems.append([rec['ACCODE'], accNames[-1], rec['P_ACCODE']])
                    items = [extendList(T, len(accNames) + 2) for T in items]

                # 저장할 위치
                valCol = accCheck[rec['ACCODE']] + 2
                
                for c in range(0, len(items)):
                    items[c][valCol] = rec[valueColumns[c]]

            # 결과 저장용 배열에 추가
            dataSet.extend(items)
        else:
            print('fetch error ', code, resp.status_code)

    # 추출한 데이터 명칭 저장
    out = open(outPath + fetchType + '-accNames.txt', 'w', encoding='utf-8')
    out.write('ACCODE' + deli + 'NAME' + deli + 'P_ACCODE\n')
    for elem in accItems:
        out.write(elem[0])
        out.write(deli)
        out.write(elem[1])
        out.write(deli)
        out.write('' if elem[2] is None else elem[2])
        out.write('\n')
        
    out.close()
    
    
    # 추출한 데이터 저장
    out = open(outPath + fetchType + '.txt', 'w', encoding='utf-8')
    
    # 컬럼명 저장
    out.write('P_CODE' + deli + 'TERM')
    
    for key in accNames:
        out.write(deli)
        out.write(key)
    out.write('\n')
    
    for rec in dataSet:
        if rec[2] is None:
           continue
        
        for i in range(0, len(rec)):
            v = rec[i]
            if i > 0: out.write(deli)
            out.write('' if v is None else str(v))
        out.write('\n')
        
    out.close()
    f.close()
    
    print(fetchType, 'done...', flush=True)


# Main Proc

# fetchFunc = { 'TestOut': fetchFinacialData }
             
fetchFunc = { 'annualBS': fetchFinacialData, 'annualCF': fetchCashFlowData, 'annualPL': fetchProfitLossData,
    'quarterBS': fetchQuaterFinacialData, 'quarterCF': fetchQuaterCashFlowData, 'quarterPL': fetchQuaterProfitLossData }

for dataType in fetchFunc.keys():
    doFetch(codePathFile, outputRawPath, dataType, fetchFunc[dataType])


In [3]:
#-*- coding:utf-8 -*-
from commonTool import *
from config import *


# Parameter 설정
set_enc_param('Ti9RYkVTZUtteTNHajA1U29qTkd3UT09')
print('enc_param', enc_param_value, flush=True)
print('codeFile', codePathFile, flush=True)


# 데이터명 분석을 위한 데이터 페치

def doFetchDataNames(referenceCodes, outPath, fetchType, funcGet):
    print(fetchType, 'doing...', flush=True)
    deli = '\t'
    # keyNames = ['ACCODE', 'ACC_NM', 'UNT_TYP', 'P_ACCODE', 'DATA1', 'DATA2', 'DATA3', 'DATA4', 'DATA5']
    valueColumns = ['DATA1', 'DATA2', 'DATA3', 'DATA4', 'DATA5']

    out = open(outPath + fetchType + '-accNames.txt', 'w', encoding='utf-8')
    out.write('PCODE' + deli + 'ACCODE' + deli + 'NAME' + deli + 'P_ACCODE\n')

    for code in referenceCodes:
        # print('fetching', code, flush=True)
        resp = funcGet(code)

        if resp.status_code == 200:
            # print(resp.text, flush=True)
            obj = json.loads(resp.text)
            if obj is None: continue
            
            data = obj['DATA']
            if data is None:
                continue

            accodeMap = {}
            
            for rec in data:
                if not rec['ACCODE'] in accodeMap:
                    accNm = rec['ACC_NM'].lstrip('.').lstrip('*')
                    accodeMap[rec['ACCODE']] = { 'name': accNm, 'pCode': rec['P_ACCODE'] }
                    
            # 추출한 데이터 명칭 저장
            
            for key in accodeMap.keys():
                elem = accodeMap[key]
                accNm = elem['name']
                
                out.write(code)
                out.write(deli)
                out.write(key)
                out.write(deli)
                out.write(accNm if elem['pCode'] is None else accodeMap[elem['pCode']]['name'] + '/' + accNm)
                out.write(deli)
                out.write('' if elem['pCode'] is None else elem['pCode'])
                out.write('\n')
            
        else:
            print('fetch error ', code, resp.status_code)

    out.close()
    print(fetchType, 'done...', flush=True)


# Main Proc

# fetchFunc = { 'TestOut': fetchFinacialData }

referenceCodes = []

fileCodes = open(codePathFile, 'r')
while True:
    line = fileCodes.readline()
    if not line:
        break

    code = line.strip()
    if len(code) <= 0:
        continue

    referenceCodes.append(code)

fileCodes.close()
    
# referenceCodes = [ '005940', '000060', '006220', '019570', '000670' ]

print('codes count: ', len(referenceCodes))

fetchFunc = { 'annualBS': fetchFinacialData, 'annualCF': fetchCashFlowData, 'annualPL': fetchProfitLossData }

for dataType in fetchFunc.keys():
    doFetchDataNames(referenceCodes, outputRawPath, dataType, fetchFunc[dataType])


enc_param MFp0eWM0Wk4yVXp6eTZWMC8xMnkydz09
codeFile D:\workspace\krx\codes.txt
codes count:  160
annualBS doing...
annualBS done...
annualCF doing...
annualCF done...
annualPL doing...
annualPL done...
